In [1]:
from load_data import load_all_feature_vectors
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import sklearn.model_selection
import keras
import mne
import numpy as np
import sklearn.model_selection
from load_data import *
import winsound
import keras
from keras import layers
from keras.models import Model
from keras.layers import Input, Dense
import keras.backend as K
from save_data_to_excel import save_data_to
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
seiz, non_seiz= load_all_feature_vectors(svm=False, all_bands=True)

Done with  1
Done with  2
Done with  3
Done with  4
Done with  5
Done with  7
Done with  9
Done with  11
Done with  14
Done with  16
Mistake Delta
Mistake Theta
Mistake Alpha
Mistake Beta
Mistake Gamma
Mistake Delta
Mistake Theta
Mistake Alpha
Mistake Beta
Mistake Gamma
Mistake Delta
Mistake Theta
Mistake Alpha
Mistake Beta
Mistake Gamma
Done with  17
Done with  18
Done with  19


In [3]:
import csv
import numpy as np
import pandas as pd
import datetime
import os
from openpyxl import load_workbook

def save_ann_data_to(data, path):
    if os.path.isfile(path):
        book = load_workbook(path)
        writer = pd.ExcelWriter(path, engine = 'openpyxl')
        writer.book = book

        time = datetime.datetime.now().strftime('%d.%m')
        df = pd.DataFrame(data, columns= ['Loss','binary_accuracy', 'precision', 'specificity'])
        df.to_excel(writer, sheet_name= 'All bands all data')

        writer.save()
        writer.close()

    else:
        time = datetime.datetime.now().strftime('%d.%m')
        df = pd.DataFrame(data,
                        columns=['Loss','binary_accuracy', 'precision', 'specificity'] )
        df.to_excel(path, sheet_name= 'All bands all data')
        
        
def save_svm_data_to(data, path):
    if os.path.isfile(path):
        book = load_workbook(path)
        writer = pd.ExcelWriter(path, engine = 'openpyxl')
        writer.book = book

        time = datetime.datetime.now().strftime('%d.%m')
        df = pd.DataFrame(data, columns= ['Mean Accuracy', 'Mean precision','Sensitivity','Specificity'])
        df.to_excel(writer, sheet_name= 'All bands all data')

        writer.save()
        writer.close()

    else:
        time = datetime.datetime.now().strftime('%d.%m')
        df = pd.DataFrame(data,
                        columns= ['Mean Accuracy', 'Mean precision','Sensitivity','Specificity'])
        df.to_excel(path, sheet_name= 'All bands all data')
        
def save_rdf_data_to(data, path):
    if os.path.isfile(path):
        book = load_workbook(path)
        writer = pd.ExcelWriter(path, engine = 'openpyxl')
        writer.book = book

        time = datetime.datetime.now().strftime('%d.%m')
        df = pd.DataFrame(data, columns= ['Mean Accuracy', 'Precision','Sensitivity','Specificity'])
        df.to_excel(writer, sheet_name= 'All bands all data')

        writer.save()
        writer.close()

    else:
        time = datetime.datetime.now().strftime('%d.%m')
        df = pd.DataFrame(data,
                        columns= ['Mean Accuracy', 'Precision','Sensitivity','Specificity'])
        df.to_excel(path, sheet_name= 'All bands all data')
        

In [30]:
#Normalize
def cut(seizure_data, non_seiz_data):
    seiz_data=[]
    for i in range(len(seizure_data)):
        if len(seizure_data[i]) == 0:
            pass
        else:
            seiz_data.append(seizure_data[i])
    
    seiz_data = np.asarray(seiz_data)
    #cut the non_seizure data so that we have a rate of 1:3 seiz:non_seiz 
    index = np.random.randint(3*len(seiz_data), len(non_seiz_data)- 3*len(seiz_data)-1)
    stop = index + int(3 * len(seiz))
    non_seiz_vector = non_seiz_data[index:stop][:]
    
    #create labels
    seiz_labels = np.ones(len(seiz_data))
    non_seiz_labels = np.zeros(len(non_seiz_vector))
    labels = np.concatenate((seiz_labels,non_seiz_labels))
    
    #concattenate the newly created vecotrs for seiz and non_seiz
    data = np.concatenate((np.asarray(seiz_data),np.asarray(non_seiz_vector)), axis=0)
    
    return normalize(data,'l1'), labels

In [8]:
#Normalize
def norm(seizure_data, non_seiz_data):
    seiz_data=[]
    for i in range(len(seizure_data)):
        if len(seizure_data[i]) == 0:
            pass
        else:
            seiz_data.append(seizure_data[i])
    
    seiz_data = np.asarray(seiz_data)
    
    #create labels
    seiz_labels = np.ones(len(seiz_data))
    non_seiz_labels = np.zeros(len(non_seiz_data))
    labels = np.concatenate((seiz_labels,non_seiz_labels))
    
    #concattenate the newly created vecotrs for seiz and non_seiz
    data = np.concatenate((np.asarray(seiz_data),np.asarray(non_seiz_data)), axis=0)
    
    return normalize(data,'l1'), labels

In [12]:
import keras.backend as K
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / ((predicted_positives) + K.epsilon())
    return precision

def specificity(y_true, y_pred):
    """
    param:
    y_pred - Predicted labels
    y_true - True labels 
    Returns:
    Specificity score
    """
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred
    fp = K.sum(neg_y_true * y_pred)
    tn = K.sum(neg_y_true * neg_y_pred)
    specificity = tn / (tn + fp + K.epsilon())
    return specificity

In [13]:
metrics = ['binary_accuracy', precision, specificity]
ann_score_hidden6 = []
ann_scores_hidden15 = []

In [14]:
for i in range(100):
    data, labels = norm(seiz.copy(),non_seiz.copy())
    seiz_x_train, seiz_x_test, seiz_y_train, seiz_y_test = sklearn.model_selection.train_test_split(data, labels)

    inp = Input(shape=(15,))
    hidden = Dense(6, activation='sigmoid')(inp)
    outp = Dense(1)(hidden)
    model= Model(inputs=inp, outputs=outp)


    model.compile(optimizer='SGD',
                  loss = 'mean_squared_error',
                 metrics=metrics) #Using MSE as lossfunction
    
    model.fit(x=seiz_x_train, y=seiz_y_train,
              batch_size=3,
              epochs=1000,
             shuffle = True,
             verbose = False)
    score = model.evaluate(x=seiz_x_test, y=seiz_y_test, verbose = False)
    ann_score_hidden6.append(score)

In [15]:
save_ann_data_to(ann_score_hidden6, 'ANN_allbands_hidden_6.xlsx')

In [16]:
for i in range(100):
    data, labels = norm(seiz.copy(),non_seiz.copy())
    seiz_x_train, seiz_x_test, seiz_y_train, seiz_y_test = sklearn.model_selection.train_test_split(data, labels)

    inp = Input(shape=(15,))
    hidden = Dense(15, activation='sigmoid')(inp)
    outp = Dense(1)(hidden)
    model= Model(inputs=inp, outputs=outp)


    model.compile(optimizer='SGD',
                  loss = 'mean_squared_error',
                 metrics=metrics) #Using MSE as lossfunction
    
    model.fit(x=seiz_x_train, y=seiz_y_train,
              batch_size=3,
              epochs=1000,
             shuffle = True,
             verbose = False)
    score = model.evaluate(x=seiz_x_test, y=seiz_y_test, verbose = False)
    ann_scores_hidden15.append(score)

In [17]:
save_ann_data_to(ann_scores_hidden15, 'ANN_allbands_hidden_15.xlsx')

In [18]:
svm_scores = []

for i in range(100):
    data, labels = norm(seiz.copy(),non_seiz.copy())
    seiz_x_train, seiz_x_test, seiz_y_train, seiz_y_test = sklearn.model_selection.train_test_split(data, labels)
    
    svm = sklearn.svm.SVC(gamma=0.5, kernel='rbf')
    svm.fit(seiz_x_train,seiz_y_train)
    
    y_pred = svm.predict(seiz_x_test)
    cm = confusion_matrix(seiz_y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    seiz_y_precision = svm.decision_function(seiz_x_test)
    average_svm_precision = sklearn.metrics.average_precision_score(seiz_y_test, seiz_y_precision)    
    svm_scores.append([svm.score(seiz_x_test,seiz_y_test), average_svm_precision, sensitivity, specificity])
    
save_svm_data_to(svm_scores,'SVM_allbands.xlsx')

In [19]:
all_rdf_ann_scores = []
for i in range(100):
    data, labels = norm(seiz.copy(),non_seiz.copy())
    seiz_x_train, seiz_x_test, seiz_y_train, seiz_y_test = sklearn.model_selection.train_test_split(data, labels)
    
    rdf = sklearn.ensemble.RandomForestClassifier()
    rdf.fit(seiz_x_train,seiz_y_train)
    y_pred = rdf.predict(seiz_x_test)
    
    
    y_pred = rdf.predict(seiz_x_test)
    rdf_cm = confusion_matrix(seiz_y_test, y_pred)

    tn, fp, fn, tp = rdf_cm.ravel()

    prec = sklearn.metrics.      (seiz_y_test, y_pred) 
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    all_rdf_ann_scores.append([rdf.score(seiz_x_test,seiz_y_test),prec,sensitivity,specificity])

save_rdf_data_to(all_rdf_ann_scores,'RDF_allbands.xlsx')


C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future

C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Till\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: Future